<a href="https://colab.research.google.com/github/novello56prod/Produzione-25/blob/main/timbrature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

sheet_name = "produzione"
sheet_id = "15-XDrVOSDiTuUnyT_FOZFkZpwZ9k4rsbkTmIHk1-qtE" # The sheet’s ID is the string located between d/ and /edit
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_timbrature = pd.read_csv(url, header=0) # Explicitly set header to 0

# Rename the first column to 'Qrcode'
df_timbrature.rename(columns={df_timbrature.columns[0]: 'Qrcode'}, inplace=True)


#df_timbrature.set_index("Qrcode", inplace=True)
#df_timbrature.sort_values(by=["Qrcode"], inplace=True, ascending=False)
df_timbrature["commessa"] = df_timbrature['Qrcode'].map(lambda x: x[0:11])
df_timbrature["durata"] = df_timbrature['Qrcode'].map(lambda x: int(x[-5:]))
df_timbrature["lavorazione"] = df_timbrature['Qrcode'].map(lambda x: x[11:13])
df_timbrature["anno"] = df_timbrature['Qrcode'].map(lambda x: int(x[3:5])+2000)
df_timbrature["posizione"] = df_timbrature['Qrcode'].map(lambda x: int(x[9:11]))
df_timbrature["numero_commessa"] = df_timbrature['Qrcode'].map(lambda x: int(x[5:9]))
# Convert 'Data_ora' to string and then extract date
df_timbrature["data"] = df_timbrature['Data_ora'].astype(str).map(lambda x: x[0:10])

#df_monitoring
valid_lavorazioni = ["AP","TP","AF","TF","TC"]

columns = ["Commessa","Anno","Numero_Commessa","Posizione","Entrata"] + valid_lavorazioni + ["Uscita"]
df_monitoring = pd.DataFrame(columns=columns)
df_monitoring.set_index("Commessa", inplace=True)


for i in range(len(df_timbrature)) :
    row_timbrature = df_timbrature.loc[i]
    qrcode = df_timbrature.loc[i]['Qrcode']
    stato = df_timbrature.loc[i]['Stato']
    addetti = df_timbrature.loc[i]['Addetti']
    nome = df_timbrature.loc[i]['Nome']
    data = df_timbrature.loc[i]['Data_ora']
    durata = df_timbrature.loc[i]['durata']
    lavorazione = df_timbrature.loc[i]['lavorazione']
    anno = df_timbrature.loc[i]['anno']
    posizione = df_timbrature.loc[i]['posizione']
    numero_commessa = df_timbrature.loc[i]['numero_commessa']
    commessa = df_timbrature.loc[i]['commessa']
    if commessa not in df_monitoring.index:
        df_monitoring.loc[commessa] = {
            "Anno": anno,
            "Numero_Commessa": numero_commessa,
            "Posizione": posizione,
            "Entrata": data,
            "AP": "-",
            "TP": "-",
            "AF": "-",
            "TF": "-",
            "TC": "-",
            "Uscita": None
        }

    if df_monitoring.loc[commessa,"Uscita"] == None:
       df_monitoring.loc[commessa,lavorazione] = stato
       for lav in valid_lavorazioni:
            df_monitoring.loc[commessa,"Uscita"] = data
            if df_monitoring.loc[commessa,lav] != 'Chiuso' and df_monitoring.loc[commessa,lav] != "-":
                df_monitoring.loc[commessa,"Uscita"] = None
                break

df_monitoring.reset_index(inplace=True)
df_monitoring.sort_values(by=["Entrata"], inplace=True, ascending=False)
df_monitoring.drop(['Commessa'], axis='columns', inplace=True)
from google.colab import data_table
#data_table.enable_dataframe_formatter()
data_table.DataTable(
    df_monitoring, include_index=False, num_rows_per_page=30
)

,Anno,Numero_Commessa,Posizione,Entrata,AP,TP,AF,TF,TC,Uscita
311,2025,3295,1,2025-08-11 8:45:22,-,x,-,x,-,None
310,2025,3257,1,2025-08-11 8:43:46,-,x,-,x,-,None
309,2025,3401,1,2025-08-11 8:43:40,-,x,-,x,x,None
308,2025,3341,1,2025-08-11 8:43:30,-,x,-,x,-,None
307,2025,2917,5,2025-08-11 8:42:44,-,x,-,x,x,None
...,...,...,...,...,...,...,...,...,...,...
276,2025,3124,2,2025-07-22 10:07:56,-,-,-,Chiuso,-,2025-07-22 10:07:56
280,2025,3037,2,2025-07-21 15:49:47,-,Chiuso,-,-,-,2025-07-21 15:49:47
281,2025,3154,1,2025-07-21 15:18:19,-,Chiuso,-,-,-,2025-07-21 15:18:19
282,2025,3152,1,2025-07-21 10:57:04,-,-,Chiuso,-,-,2025-07-21 9:38:10


In [ ]:
df_carico = df_timbrature[df_timbrature["Stato"]=="x"].groupby(["data","lavorazione"])['durata'].sum()
df_scarico = df_timbrature[df_timbrature["Stato"]=="Chiuso"].groupby(["data","lavorazione"])['durata'].sum()
df_wip = pd.merge(df_carico, df_scarico, on=['data', 'lavorazione'], how='outer')
df_wip.rename(columns={'durata_x': 'carico', 'durata_y': 'scarico'}, inplace=True)
df_wip.fillna(0, inplace=True)
df_wip["saldo"] = df_wip["carico"] - df_wip["scarico"]
df_wip["OLE"] = df_wip['scarico'].map(lambda x: str(int(x/480*100))+"%")
df_wip.sort_values(by=["data"], inplace=True, ascending=False)
print(df_wip)

                        carico  scarico   saldo   OLE
data       lavorazione                               
2025-08-07 TP            353.0     52.0   301.0   10%
           TF            829.0    163.0   666.0   33%
           TC            205.0     62.0   143.0   12%
           AP             31.0      0.0    31.0    0%
2025-08-06 TP            165.0     37.0   128.0    7%
           TF            554.0    611.0   -57.0  127%
           TC            177.0    132.0    45.0   27%
2025-08-05 TP             19.0    670.0  -651.0  139%
           TF            101.0    718.0  -617.0  149%
           TC             74.0    398.0  -324.0   82%
2025-08-04 TF           1714.0    501.0  1213.0  104%
           TC            629.0    384.0   245.0   80%
           TP            674.0    430.0   244.0   89%
2025-08-01 TP           1033.0    264.0   769.0   55%
           TF           1688.0    374.0  1314.0   77%
           TC              0.0    188.0  -188.0   39%
2025-07-31 TP            350